In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool, Range1d
from scipy import stats
import bokeh_catplot
import pandas as pd
import math

from bokeh.layouts import row
bokeh.io.output_notebook()

from tkinter import Tk
from tkinter.filedialog import askdirectory

Loading BokehJS ...

In [2]:
def upgradeCSV(data_dir, app_root_dir, save_folder, filename_distinct_substring, TechRep):
    extension = 'csv'
    list_csv = []
    for root, dirs, files in os.walk(data_dir, topdown = False):
        for name in files:
            if extension in name:
                if filename_distinct_substring in name:
                    list_csv.append(os.path.join(root, name))
    os.chdir(app_root_dir)
    app_data_dir = "new_data\\"
    if not os.path.exists(app_data_dir):
        os.mkdir(app_data_dir)
    os.chdir(app_data_dir)
    new_files_dir = save_folder
    if not os.path.exists(new_files_dir):
        os.mkdir(new_files_dir)
    os.chdir(new_files_dir)
    for file in list_csv:
        df_temp = pd.read_csv(file, sep = ',', na_values = '*')
        names = file.rsplit('\\',1)[1].rsplit('_',4)
        df_temp['Strain'] = names[0]
        df_temp['Reporter'] = names[1]
        df_temp['TimePoint'] = names[2].rsplit('h',1)[0]
        df_temp['TeckRep'] = TechRep
        df_temp.to_csv(file.rsplit('\\',1)[1], index = None, header = True)
        
def msd(xdata, ydata):
    r=np.sqrt(xdata**2+ydata**2)
    diff=np.diff(r)
    diff_sq=diff**2
    MSD=np.mean(diff_sq)
    return MSD

def rmsd(xdata, ydata):
    xdata_x0=np.subtract(xdata, xdata[0])
    x=xdata_x0[1:]
    ydata_y0=np.subtract(ydata, ydata[0])
    y=ydata_y0[1:]
    RMSD=np.sqrt(np.mean(x**2+y**2))
    return RMSD

def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians, stats.sem(bs_medians)

In [3]:
#set root folder
data_dir = "F:\\Sauvegardes PhD\\Covid-19\\Chp_mutants_pili\\Dynamics_of_Pili\\pilG-cpdA-\\"
app_root_dir = "C:\\Users\\tala\\Desktop\\git\\PhD_codes\\Mechanosensation\\Python_code\\Pole_analysis\\PiliDynamics\\"

new_dir = "pilG-cpdA-fliC-_Data\\"
if not os.path.exists(app_root_dir+new_dir):
    os.mkdir(app_root_dir+new_dir)

os.chdir(data_dir)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(data_dir, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(os.path.join(root, name))
print(len(list_csv))

combined_csv = pd.concat([pd.read_csv(f) for f in list_csv ], sort=False)

os.chdir(app_root_dir + new_dir)
combined_csv.to_csv( "combined_pilidynamics_Data.csv", index=False, encoding='utf-8-sig')
combined_csv.head()

5


,,Label,Area,Mean,StdDev,Min,Max,Median,Slice,Cell_ID,...,StdPoleBright,Nb_Pili_PoleBright,Nb_Flagella_PoleBright,PolarRatio,RetractedPilusLength_Bright,RetractedPilusFrame_Bright,RetractedPilusLength_Dim,RetractedPilusFrame_Dim,X,Y
0,1,pilG-cpdA-fliC-_Gasket_0_event11_tirf_RAW_Stac...,974,255,0.0,255,255,255.0,1.0,1,...,129.753,1,0,1.015,118.849,71,NaN,NaN,NaN,NaN
0,1,pilG-cpdA-fliC-_Gasket_0_event13_tirf_RAW_Stac...,1355,255,0.0,255,255,255.0,1.0,1,...,73.346,0,0,1.012,0.000,0,45.525,478.0,NaN,NaN
1,2,pilG-cpdA-fliC-_Gasket_0_event13_tirf_RAW_Stac...,0,0,0.0,0,0,0.0,0.0,1,...,73.346,0,0,0.000,0.000,0,40.311,496.0,NaN,NaN
2,3,pilG-cpdA-fliC-_Gasket_0_event13_tirf_RAW_Stac...,0,0,0.0,0,0,0.0,0.0,1,...,73.346,0,0,0.000,0.000,0,31.753,509.0,NaN,NaN
3,4,pilG-cpdA-fliC-_Gasket_0_event13_tirf_RAW_Stac...,0,0,0.0,0,0,0.0,0.0,1,...,73.346,0,0,0.000,0.000,0,24.083,529.0,NaN,NaN


In [4]:
Labels=combined_csv['Label'].unique()
Labels

array(['pilG-cpdA-fliC-_Gasket_0_event11_tirf_RAW_Stack.tif',
       'pilG-cpdA-fliC-_Gasket_0_event13_tirf_RAW_Stack.tif',
       'pilG-cpdA-fliC-_Gasket_0_event18_tirf_RAW_Stack.tif',
       'PilG-CpdA-FliC-_3_Microchannel_2_event1_tirf_RAW_Stack.tif',
       'pilG-cpdA-fliC-_Gasket_0_event8_tirf_RAW_Stack.tif'], dtype=object)

In [5]:
cells=combined_csv.loc[combined_csv['Label']==Labels[1], 'Cell_ID'].unique()
cells

array([1, 2], dtype=int64)

In [6]:
N_cells=0
for label in Labels:
    Cells=combined_csv.loc[combined_csv['Label']==label, 'Cell_ID'].unique()
    N_cells=N_cells+len(Cells)
    for cell in Cells:
        Raw=combined_csv.loc[(combined_csv['Label']==label) & (combined_csv['Cell_ID']==cell), 'RetractedPilusFrame_Bright']
        Do_not_keep = np.isnan(Raw)
        Remove_nan = ~Do_not_keep
        Retraction_NaN=Raw[Remove_nan]
        Bool=np.ones((len(Retraction_NaN),1)).astype(bool)
        print(Bool)
        zeros=np.where(Retraction_NaN == 0)
        for i in zeros:
            Bool[i]=False
        print(Bool)
        Retraction=Retraction_NaN[Bool]
        del zeros, Do_not_keep, Remove_nan, Raw, Retraction_NaN, Retraction
print(N_cells)

[[ True]]
[[ True]]


IndexError: too many indices for array

In [38]:
array1 = np. array([np. nan, 1, 0, 2, 0])
print(array1)
nan_array = np. isnan(array1)
not_nan_array = ~ nan_array
array2 = array1[not_nan_array]
print(array2)

[nan  1.  0.  2.  0.]
[1. 0. 2. 0.]


In [21]:
zeros=np.where(array1 == 0)
print(zeros)
np.delete(array1,zeros)

(array([2, 4], dtype=int64),)


array([nan,  1.,  2.])

In [41]:
del zeros, Do_not_keep, Remove_nan, Raw, Retraction_NaN, Retraction

NameError: name 'Retraction' is not defined

In [9]:
Remove_zero=~zeros
        if len(zeros) < len(Raw):
            Retraction=Retraction_NaN[Remove_zero]
        else:
            Retraction=Retraction_NaN
        print(Retraction)

NameError: name 'Remove_zero' is not defined

In [14]:
zeros=np.where(Retraction_NaN == 0)
zeros
Do_not_keep

0    False
Name: RetractedPilusLength_Bright, dtype: bool

In [13]:
a=[False, False, False]

In [14]:
print(a)

[False, False, False]
